## Here I focus strictly on the problem with multple layers

In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers
import conv_models
import tensorflow.keras.backend as K
import keras
from keras.layers import Input, Dense
from keras.models import Model

Using TensorFlow backend.


In [2]:
adj = np.load('/Users/b_eebs/tf-keras/ZINC/adj/0.npy').astype('float32')
features = np.load('/Users/b_eebs/tf-keras/ZINC/features/0.npy').astype('float32')
test = (np.load('/Users/b_eebs/tf-keras/ZINC/logP.npy')[0:10000]).astype('float32') #takes the first 10,000 molecules

In [3]:
class convLayer(layers.Layer):

    def __init__(self, output_dim, adjacency, batch_size, **kwargs):
        self.output_dim = output_dim
        self.adjacency = adjacency # adj shape is [should be batch size , 50, 50]
        self.batch_size = batch_size # hyper parameter 
        super(convLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        print('in shape',input_shape)
        shape = tf.TensorShape((input_shape[1], self.output_dim))
        shape = [int(shape[0]),int(shape[1])] # [50 , 32]
        self.kernel = self.add_weight(name='kernel',
                                      shape=shape,
                                      initializer='glorot_uniform',
                                      trainable=True,dtype=tf.float32)

        self.bias = self.add_weight(name='bias',
                            shape=[shape[1]],
                            initializer='glorot_uniform',
                            trainable=True,dtype=tf.float32)

        super(convLayer, self).build(input_shape)

    def call(self, inputs):
        _features = conv_models.GCN(inputs,self.adjacency,self.kernel,self.bias) #[ batch_size, 50, 32]
        _features = tf.reshape(_features, [batch_size, -1]) # [batch_size 1600], fl. 32
        _features = tf.cast(_features, tf.float32) # [batch_size 1600], fl. 64
        return _features
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


In [4]:
# I think what's happening is that the output from the first layer is directly going into the second layer
# flat
# confirmed, this is happening.

conv_output = 32
K.set_learning_phase(1)
batch_size = 1
num_layers = 6
target_shape = (50, 32)
model = tf.keras.Sequential()
for i in range(num_layers):
    model.add(convLayer(32,adj[:1],batch_size))
    model.add(tf.keras.layers.Reshape(target_shape))
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mse',
              metrics=['mae'])
model.fit(features[:1], test[:1],batch_size= batch_size,epochs = 3, steps_per_epoch = 3)
model.summary()
(model.inputs)



Instructions for updating:
Colocations handled automatically by placer.
in shape (?, 50, 58)
Tensor("input_1:0", shape=(?, 50, 58), dtype=float32)
<tf.Variable 'conv_layer/kernel:0' shape=(50, 32) dtype=float32>


ValueError: Cannot reshape a tensor with 1600 elements to shape [58,32] (1856 elements) for 'einsum/Reshape_1' (op: 'Reshape') with input shapes: [50,32], [2] and with input tensors computed as partial shapes: input[1] = [58,32].